In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# Load YOLO
yolo = cv2.dnn.readNet("C:/Users/ADMIN/Desktop/Object Detection/yolov3-tiny.weights", "C:/Users/ADMIN/Desktop/Object Detection/yolov3-tiny.cfg")

NameError: name 'cv2' is not defined

In [3]:
# Load class names
classes = []
with open("C:/Users/ADMIN/Desktop/Object Detection/coco.names", 'r') as f:
    classes = f.read().splitlines()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/projects/Object Detection/coco.names'

In [5]:
# Initialize webcam
cap = cv2.VideoCapture(0)

In [6]:
while True:
    ret, frame = cap.read()
    height, width, channels = frame.shape
    
    # Preprocess the image
    blob = cv2.dnn.blobFromImage(frame, 1/255, (320, 320), (0, 0, 0), swapRB=True, crop=False)
    yolo.setInput(blob)
    output_layer_names = yolo.getUnconnectedOutLayersNames()
    layer_outputs = yolo.forward(output_layer_names)
    
    # Initialization
    boxes = []
    confidences = []
    class_ids = []
    
    # Extract bounding boxes and confidences
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.7:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))
    
    if isinstance(indexes, tuple):
        indexes = np.array(indexes)  # Convert tuple to numpy array if necessary

    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confi = str(round(confidences[i], 2))
        color = colors[i]
        
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 3)
        cv2.putText(frame, label + " " + confi, (x, y + 20), font, 2, (255, 255, 255), 3)
    
    # Display the image
    cv2.imshow('Image', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 